In [7]:
import numpy as np
import matplotlib.pyplot as plt
from math import floor

file_pdcp_dl = "/home/luca/Scaricati/output/prova/mcDlPdcpStats_5_10_22_12_2017_12_29_53.txt"

n_ues = 12;
ues_rx = [[] for _ in range(n_ues)]
time_quantum = 0.01
start_time = 0
end_time = 60# da impostare

with open(file_pdcp_dl, 'r') as f: 
    prev_line = f.readline()
    for line in f:
        # elimino duplicati consecutivi
        if (line != prev_line):
            tokens = line.rstrip().split(' ')
            if tokens[0] == "Rx":
                lcid = int(tokens[5])
                if lcid > 1:
                    time = float(tokens[1])
                    ue = int(tokens[3]) - 1
                    lcid = int(tokens[5])
                    pck_size = int(tokens[6]) / (10e6 * time_quantum)
                    pck_delay = int(tokens[7]) / 10e6
                    ues_rx[ue].append((time, pck_size, pck_delay))
    f.close()

times = np.arange(start_time, end_time + time_quantum, time_quantum)
throughput = np.zeros((n_ues, len(times)))
avg_throughput = np.zeros(n_ues)
delay = np.zeros((n_ues, len(times)))
avg_delay = np.zeros(n_ues)

for i in range(n_ues):
    count = np.zeros(len(times))
    for pck in ues_rx[i]:
        throughput[i, int(floor(pck[0] / time_quantum))] += pck[1]
        avg_throughput[i] += pck[1] / len(times)
        delay[i, int(floor(pck[0] / time_quantum))] += pck[2]
        count[int(floor(pck[0] / time_quantum))] += 1
    count = count.clip(min = 1)
    delay[i] = np.divide(delay[i], count)
    avg_delay[i] = np.mean(delay[i])

#sigma_delay = [np.sqrt(np.mean(np.square(delay[i] - mean_delay[i]))) for i in range(n_ues)]

for i in range(n_ues):
    fig = plt.figure(figsize = (20, 10))
    ax1 = plt.gca()
    ax1.set_xlim(start_time, end_time)
    ax1.xaxis.set_ticks(np.arange(start_time, end_time, time_quantum))
    ax1.xaxis.set_ticklabels(np.arange(start_time, end_time, time_quantum))

    plt.subplot(2, 1, 1)
    plt.title("Throughput ue" + str(i + 1))
    plt.xlabel("Time [s]")
    plt.ylabel('Througput [Mb/s]')
    plt.grid(True)

    plt.plot(times, throughput[i], lw = 2, color = 'b')
    plt.plot(times, np.ones(len(times)) * avg_throughput[i], '--', lw = 2, color = 'b')
    #plt.plot(times, np.ones(len(times)) * sigma_throughput[i], '--', lw = 2, color = 'g')
    
    plt.legend(['throughput', 'mean'])
    
    plt.subplot(2, 1, 2)
    plt.title("Delay ue" + str(i + 1))
    plt.xlabel("Time [s]")
    plt.ylabel('Delay [ms]')
    plt.grid(True)

    plt.plot(times, delay[i], lw = 2, color = '#800080')
    plt.plot(times, np.ones(len(times)) * avg_delay[i], '--', lw = 2, color = '#800080')
    #plt.plot(times, np.ones(len(times)) * sigma_delay[i], '--', lw = 2, color = 'r')
    
    plt.legend(['delay', 'mean'])
    
    plt.tight_layout()
    plt.show()

KeyboardInterrupt: 